<a href="https://colab.research.google.com/github/IlyaTsibulin/colab-tryon/blob/main/StableVITON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/rlawjdghek/StableVITON.git
%cd StableVITON

Cloning into 'StableVITON'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 143 (delta 16), reused 122 (delta 13), pack-reused 1
Receiving objects: 100% (143/143), 2.43 MiB | 12.65 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/StableVITON


In [ ]:
!pip uninstall -y torch torchvision torchaudio torchtext
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchtext==0.14.0
!pip install pytorch-lightning==1.5.0 einops opencv-python matplotlib omegaconf albumentations transformers==4.33.2
!pip install xformers==0.0.19 triton==2.0.0 open-clip-torch==2.19.0 diffusers==0.20.2 scipy==1.10.1

Found existing installation: torch 2.0.0
Uninstalling torch-2.0.0:
  Successfully uninstalled torch-2.0.0
Found existing installation: torchvision 0.15.1
Uninstalling torchvision-0.15.1:
  Successfully uninstalled torchvision-0.15.1
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Using cached torchvision-0.14.1-cp310-cp310-manylinux1_x86_64.whl (24.2 MB)
  Using cached torchaudio-0.13.1-cp310-cp310-manylinux1_x86_64.whl (4.2 MB)
  Using cached torchtext-0.14.0-cp310-cp310-manylinux1_x86_64.whl (2.0 MB)
INFO: pip is looking at multiple versions of torchtext to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==1.13.1, torchaudio==0.13.1, torchtext==0.14.0 and torchvision==0.14.1 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==1.13.1
    torchvision 0.14.1 depends on torch==1.13.1
    torchaudio 0.13.1 depends on torch==1.13.

In [ ]:
# Create the checkpoints directory
!mkdir -p ckpts

# Download the pre-trained model weights
!wget -O ckpts/VITONHD_PBE_pose.ckpt https://huggingface.co/rlawjdghek/StableVITON/blob/main/ckpts/VITONHD_PBE_pose.ckpt
!wget -O ckpts/VITONHD_VAE_finetuning.ckpt https://huggingface.co/rlawjdghek/StableVITON/blob/main/ckpts/VITONHD_VAE_finetuning.ckpt

--2024-07-13 13:35:24--  https://huggingface.co/rlawjdghek/StableVITON/blob/main/ckpts/VITONHD_PBE_pose.ckpt
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.88, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39344 (38K) [text/html]
Saving to: ‘ckpts/VITONHD_PBE_pose.ckpt’

ckpts/VITONHD_PBE_p 100%[===================>]  38.42K  --.-KB/s    in 0.01s   

2024-07-13 13:35:24 (3.92 MB/s) - ‘ckpts/VITONHD_PBE_pose.ckpt’ saved [39344/39344]

--2024-07-13 13:35:25--  https://huggingface.co/rlawjdghek/StableVITON/blob/main/ckpts/VITONHD_VAE_finetuning.ckpt
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.88, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43635 (43K) [text/html]
Saving to: ‘ckpts/VITONHD_VAE_finetuning.ckpt’

ckpts/VITONHD_VAE_f 

In [ ]:
!mkdir -p test/image
!mkdir -p test/cloth
!mkdir -p test/image-densepose
!mkdir -p test/agnostic
!mkdir -p test/agnostic-mask
!mkdir -p test/cloth_mask

In [ ]:
from google.colab import files


print("Upload test images")
uploaded_images = files.upload()
for filename in uploaded_images.keys():
    !mv {filename} test/image/

print("Upload clothing images")
uploaded_cloths = files.upload()
for filename in uploaded_cloths.keys():
    !mv {filename} test/cloth/

Upload test images


Upload clothing images


In [ ]:
import os

image_files = os.listdir('test/image')
cloth_files = os.listdir('test/cloth')

print(f"Number of test images: {len(image_files)}")
print(f"Number of cloth images: {len(cloth_files)}")

if len(image_files) != len(cloth_files):
    print("Error: The number of image files and cloth files must be the same.")
else:
    with open('test/test_pairs.txt', 'w') as f:
        for image_file, cloth_file in zip(image_files, cloth_files):
            f.write(f"{image_file} {cloth_file}\n")

Number of test images: 1
Number of cloth images: 1


In [ ]:
!python /content/StableVITON/inference.py --config_path ./StableVITON/configs/VITONHD.yaml --batch_size 4 --model_load_path ./ckpts/VITONHD_PBE_pose.ckpt --save_dir result